## 📦 필요한 라이브러리 설치

In [ ]:
!pip install --upgrade google-api-python-client pandas google-auth-oauthlib google-auth-httplib2

## ✅ 분석 코드

In [ ]:
# -*- coding: utf-8 -*-
"""
YouTube 댓글 수집 스크립트 (Colab용)

지정한 검색어(제품명 등)로 YouTube 영상을 검색하고,
검색된 인기 영상들로부터 댓글 데이터를 수집.
"""

# 2. 라이브러리 임포트
import os
import pandas as pd
from google.colab import userdata # Colab의 Secrets 기능 사용
import googleapiclient.discovery
import googleapiclient.errors

# 3. YouTube API 키 설정
# !! 중요 !!
# Colab 좌측 메뉴의 'Secrets'(열쇠 모양 아이콘)에
# 'YOUTUBE_API_KEY' 라는 이름으로 자신의 YouTube Data API v3 키를 저장해서 진행
# 즉 이거 따로 돌릴 때 자기껄로 발급 받아서 넣어야혀요
try:
    API_KEY = userdata.get('YOUTUBE_API_KEY')
    if not API_KEY:
        raise ValueError("API Key not found in Colab Secrets.")
    print("YouTube API 키를 성공적으로 로드했습니다.")
except Exception as e:
    print(f"오류: Colab Secrets에서 'YOUTUBE_API_KEY'를 찾을 수 없습니다. {e}")
    print("Colab 좌측 메뉴 'Secrets'에 API 키를 추가해주세요.")
    # 스크립트 실행 중단 또는 기본값 설정 등의 처리가 필요할 수 있음
    API_KEY = None # 예외 발생 시 API_KEY를 None으로 설정

# 4. YouTube API 클라이언트 빌드 함수
def get_youtube_client(api_key):
    """YouTube API 서비스 클라이언트를 생성합니다."""
    if not api_key:
        print("오류: API 키가 유효하지 않아 YouTube 클라이언트를 생성할 수 없습니다.")
        return None
    api_service_name = "youtube"
    api_version = "v3"
    try:
        youtube = googleapiclient.discovery.build(
            api_service_name, api_version, developerKey=api_key)
        return youtube
    except Exception as e:
        print(f"YouTube 클라이언트 생성 중 오류 발생: {e}")
        return None

# 5. 동영상 검색 함수
def search_videos(youtube, query, max_results=5):
    """주어진 쿼리로 동영상을 검색하고 videoId 목록을 반환합니다."""
    if not youtube:
        return []
    try:
        request = youtube.search().list(
            part="snippet",
            q=query,
            type="video",
            order="relevance", # 관련성 순서 (viewCount: 조회수 순)
            maxResults=max_results
        )
        response = request.execute()

        videos = []
        for item in response.get('items', []):
            video_id = item.get('id', {}).get('videoId')
            video_title = item.get('snippet', {}).get('title')
            if video_id and video_title:
                videos.append({'video_id': video_id, 'video_title': video_title})
        print(f"'{query}' 검색 결과: {len(videos)}개의 관련 영상을 찾았습니다.")
        return videos
    except googleapiclient.errors.HttpError as e:
        print(f"동영상 검색 중 API 오류 발생: {e}")
        if e.resp.status == 403:
            print("API 할당량 초과 또는 권한 문제가 발생했을 수 있습니다.")
        return []
    except Exception as e:
        print(f"동영상 검색 중 예상치 못한 오류 발생: {e}")
        return []


# 6. 특정 동영상의 댓글 수집 함수
def get_video_comments(youtube, video_id, max_comments_per_video=100):
    """주어진 videoId의 댓글을 수집합니다."""
    if not youtube:
        return []

    comments = []
    next_page_token = None
    comments_fetched = 0

    print(f"  [Video ID: {video_id}] 댓글 수집 시작 (최대 {max_comments_per_video}개)...")

    while comments_fetched < max_comments_per_video:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=min(100, max_comments_per_video - comments_fetched), # 페이지당 최대 100개
                pageToken=next_page_token,
                textFormat="plainText" # HTML 태그 제거
            )
            response = request.execute()

            for item in response.get("items", []):
                if comments_fetched >= max_comments_per_video:
                    break

                comment_snippet = item.get("snippet", {}).get("topLevelComment", {}).get("snippet", {})
                comment_id = item.get("snippet", {}).get("topLevelComment", {}).get("id")
                comment_text = comment_snippet.get("textDisplay")
                author = comment_snippet.get("authorDisplayName", "Unknown")
                published_at = comment_snippet.get("publishedAt")

                if comment_text:
                    comments.append({
                        'comment_id': comment_id,
                        'comment_text': comment_text,
                        'comment_author': author,
                        'comment_published_at': published_at
                    })
                    comments_fetched += 1

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                # print(f"  [Video ID: {video_id}] 더 이상 댓글이 없습니다.")
                break # 다음 페이지 없으면 종료

        except googleapiclient.errors.HttpError as e:
            if 'commentsDisabled' in str(e):
                print(f"  [Video ID: {video_id}] 댓글 기능이 비활성화된 영상입니다.")
            elif e.resp.status == 403:
                 print(f"  [Video ID: {video_id}] 댓글 접근 권한 문제 또는 API 할당량 초과 가능성이 있습니다.")
            else:
                print(f"  [Video ID: {video_id}] 댓글 수집 중 API 오류 발생: {e}")
            return comments # 오류 발생 시 현재까지 수집된 댓글만 반환
        except Exception as e:
            print(f"  [Video ID: {video_id}] 댓글 수집 중 예상치 못한 오류 발생: {e}")
            return comments # 오류 발생 시 현재까지 수집된 댓글만 반환

    print(f"  [Video ID: {video_id}] 댓글 {comments_fetched}개 수집 완료.")
    return comments

YouTube API 키를 성공적으로 로드했습니다.


In [ ]:
# 7. 메인 실행 로직
if __name__ == "__main__":
    if API_KEY: # API 키가 유효할 때만 실행
        youtube_client = get_youtube_client(API_KEY)

        if youtube_client:
            # --- 사용자 입력 ---
            search_query = input("댓글을 수집할 제품명을 입력하세요 (예: CU 밤티라미수): ")
            num_videos_to_search = int(input("검색할 관련 영상의 최대 개수를 입력하세요 (예: 5): "))
            num_comments_per_video = int(input("각 영상에서 수집할 최대 댓글 수를 입력하세요 (예: 100): "))
            # -----------------

            print(f"\n'{search_query}' 관련 영상 검색 및 댓글 수집을 시작합니다...")

            # 1. 관련 영상 검색
            videos_found = search_videos(youtube_client, search_query, num_videos_to_search)

            import time  # 이미 되어있다면 생략

            all_comments_data = []
            start_time = time.time()  # ▶ 수집 시작 시각 기록


            all_comments_data = []

            if videos_found:
                # 2. 각 영상별 댓글 수집
                for video in videos_found:
                    video_id = video['video_id']
                    video_title = video['video_title']
                    print(f"\n영상 제목: {video_title} (ID: {video_id})")

                    comments = get_video_comments(youtube_client, video_id, num_comments_per_video)

                    # 수집된 댓글에 video_id와 video_title 정보 추가
                    for comment in comments:
                        comment['video_id'] = video_id
                        comment['video_title'] = video_title

                    all_comments_data.extend(comments)

                print(f"\n총 {len(all_comments_data)}개의 댓글을 수집했습니다.")
                end_time = time.time()
                elapsed = end_time - start_time
                print(f"⏱️ 댓글 수집 소요 시간: {elapsed:.2f}초")

                # 3. 결과를 Pandas DataFrame으로 변환
                if all_comments_data:
                    df_comments = pd.DataFrame(all_comments_data)

                    # 보기 좋게 컬럼 순서 조정
                    df_comments = df_comments[['video_id', 'video_title', 'comment_id', 'comment_published_at', 'comment_author', 'comment_text']]

                    print("\n--- 수집된 댓글 데이터 샘플 ---")
                    print(df_comments.head())

                    # 4. (선택) 결과를 CSV 파일로 저장
                    save_csv = input("\n수집된 댓글 데이터를 CSV 파일로 저장하시겠습니까? (y/n): ")
                    if save_csv.lower() == 'y':
                        csv_filename = f"{search_query.replace(' ', '_')}_comments.csv"
                        df_comments.to_csv(csv_filename, index=False, encoding='utf-8-sig')
                        print(f"'{csv_filename}'으로 저장되었습니다.")
                else:
                    print("\n수집된 댓글 데이터가 없습니다.")
            else:
                print(f"'{search_query}'에 대한 관련 영상을 찾지 못했습니다.")
        else:
            print("YouTube API 클라이언트 초기화에 실패했습니다.")
    else:
        print("YouTube API 키가 설정되지 않아 스크립트를 실행할 수 없습니다.")

댓글을 수집할 제품명을 입력하세요 (예: CU 밤티라미수): CU밤티라미수
검색할 관련 영상의 최대 개수를 입력하세요 (예: 5): 20
각 영상에서 수집할 최대 댓글 수를 입력하세요 (예: 100): 200

'CU밤티라미수' 관련 영상 검색 및 댓글 수집을 시작합니다...
'CU밤티라미수' 검색 결과: 20개의 관련 영상을 찾았습니다.

영상 제목: CU밤티라미수컵🌰 (ID: jWrZjXaozIg)
  [Video ID: jWrZjXaozIg] 댓글 수집 시작 (최대 200개)...
  [Video ID: jWrZjXaozIg] 댓글 4개 수집 완료.

영상 제목: 나폴리 맛피아 CU 밤티라미수 후기 죄송하지만 보류입니다.. (ID: tgntgIgXvG0)
  [Video ID: tgntgIgXvG0] 댓글 수집 시작 (최대 200개)...
  [Video ID: tgntgIgXvG0] 댓글 32개 수집 완료.

영상 제목: 백종원 &amp; 안성재 함박웃음 짓게 만든 나폴리 맛피아의 밤 티라미수 | 흑백요리사: 요리 계급 전쟁 | 넷플릭스 (ID: g3hVg98AMlI)
  [Video ID: g3hVg98AMlI] 댓글 수집 시작 (최대 200개)...
  [Video ID: g3hVg98AMlI] 댓글 200개 수집 완료.

영상 제목: CU 밤티라미수 피드백 중인 맛피아 (ID: 4x6UzeY5vSM)
  [Video ID: 4x6UzeY5vSM] 댓글 수집 시작 (최대 200개)...
  [Video ID: 4x6UzeY5vSM] 댓글 100개 수집 완료.

영상 제목: 나폴리 맛피아님 밤티라미수 개선판이 나왔습니다 (ID: VOL3vIPk3Ec)
  [Video ID: VOL3vIPk3Ec] 댓글 수집 시작 (최대 200개)...
  [Video ID: VOL3vIPk3Ec] 댓글 80개 수집 완료.

영상 제목: 드디어 흑백요리사 CU 밤 티라미수 영접 솔직후기? (ID: oufY5SKulsU)
  [Video ID: oufY5SKulsU] 댓